In [ ]:
import { document } from 'jsr:@ry/jupyter-helper';
import pl from 'npm:nodejs-polars';
import * as Plot from 'npm:@observablehq/plot';
let { html, md, display } = Deno.jupyter;


In [ ]:
type Seriesify<T extends Record<string, pl.DataType>> = {
  [key in keyof T]: key extends string ? pl.Series<T[key], key> : pl.Series<T[key]>;
};


In [ ]:
type Ascent = {
  created_at: pl.Datetime;
  ascendable_id: pl.Int64;
  ascendable_name: pl.String;
  ascendable_type: pl.Categorical; //'GymBoulder';
  grading_system: pl.Categorical; //'font';
  grade: pl.String;
  difficulty: pl.String;
  type: pl.Categorical; // 'rp' | 'f';
  sub_type: pl.String;
  tries: pl.Int64;
};

let ascents = pl.readJSON('./data/ascents.jsonl', {
  inferSchemaLength: null,
  format: 'lines',
}) as pl.DataFrame<Seriesify<Ascent>>;

ascents = ascents
  .withColumns(
    ascents.getColumn('created_at').str.strptime(pl.Datetime, '%Y-%m-%d %H:%M:%S %z').cast(pl.Datetime('ms')),
    ascents.getColumn('updated_at').str.strptime(pl.Datetime, '%Y-%m-%d %H:%M:%S %z').cast(pl.Datetime('ms')),
    ascents.getColumn('ascendable_type').cast(pl.Categorical),
    ascents.getColumn('grading_system').cast(pl.Categorical),
  )
  .withColumns(pl.col('created_at').date.year().alias('year'))
  // remove repeat ascents
  .unique({ subset: 'ascendable_id', keep: 'first' });


In [ ]:
type Boulder = {
  id: pl.Int64;
  route_setter: pl.String;
  sector_name: pl.String;
  gym_wall_name: pl.String;
  set_at: pl.Datetime;
  route_card_label: pl.String;
};

let boulders = pl
  .readJSON('./data/gym_boulders.jsonl', {
    inferSchemaLength: null,
    format: 'lines',
  })
  .withColumns(
    pl.col('route_card_label').str.slice(0, 3),
    pl.col('set_at').str.strptime(pl.Datetime, '%Y-%m-%d %H:%M:%S %z').cast(pl.Datetime('ms')),
    pl.col('id').alias('nid'),
  ) as pl.DataFrame<Seriesify<Boulder>>;


In [ ]:
let detailedAscents = ascents
  .join(boulders, {
    leftOn: 'ascendable_id',
    rightOn: 'nid',
  })
  .sort(pl.col('created_at'));


In [ ]:
let boulderScores = [
  { score: 12, v: 'VB', font: '2' },
  { score: 18, v: 'VB', font: '2+' },
  { score: 24, v: 'VB+', font: '3' },
  { score: 30, v: 'VB+', font: '3+' },
  { score: 36, v: 'V0-', font: '4' },
  { score: 42, v: 'V0', font: '4+' },
  { score: 48, v: 'V1', font: '5' },
  { score: 54, v: 'V2', font: '5+' },
  { score: 60, v: 'V3', font: '6a' },
  { score: 66, v: 'V3', font: '6a+' },
  { score: 72, v: 'V4', font: '6b' },
  { score: 78, v: 'V4', font: '6b+' },
  { score: 84, v: 'V5', font: '6c' },
  { score: 90, v: 'V5', font: '6c+' },
  { score: 96, v: 'V6', font: '7a' },
  { score: 102, v: 'V7', font: '7a+' },
  { score: 108, v: 'V7', font: '7b' },
  { score: 114, v: 'V8', font: '7b+' },
  { score: 120, v: 'V9', font: '7c' },
  { score: 126, v: 'V10', font: '7c+' },
].map((grade) => ({ ...grade, font: grade.font.toUpperCase() }));
let gradeToNumber = boulderScores.reduce((acc, grade) => {
  acc[grade.font] = grade.score;
  return acc;
}, {});

gradeToNumber;


{
  "2": 12,
  "3": 24,
  "4": 36,
  "5": 48,
  "2+": 18,
  "3+": 30,
  "4+": 42,
  "5+": 54,
  "6A": 60,
  "6A+": 66,
  "6B": 72,
  "6B+": 78,
  "6C": 84,
  "6C+": 90,
  "7A": 96,
  "7A+": 102,
  "7B": 108,
  "7B+": 114,
  "7C": 120,
  "7C+": 126
}

In [ ]:
detailedAscents = detailedAscents.withColumns(
  pl.col('difficulty').replace({ old: gradeToNumber }).cast(pl.Float32).alias('numberDifficulty'),
  pl.col('type').replace(['f', 'rp'], ['flash', 'redpoint']),
);


In [ ]:
html`${detailedAscents.getColumn('route_setter').valueCounts().sort('count', true).toHTML()}`;


route_setter,count
Anton Cuevas,254
Silver Breiktopf,175
Benjamin Schlobohm,153
James Thanawat,137
Stina Dorwarth,95
N.N.,73
Sebastian Dahlgren,71
Robert Rundin,51
Isabelle Fugelstad,19
Sophie Machaczek,16


In [ ]:
let baseWidth = 640;
let ascentTypeColorDomain = { domain: ['flash', 'redpoint'], range: ['#ffd800', '#ff5151'] };


In [ ]:
let groupings = [
  {
    name: 'Everything',
    y: 'difficulty',
    fill: 'type',
  },
  {
    name: 'By year',
    y: 'difficulty',
    fy: 'year',
    fill: 'type',
  },
  {
    name: 'By route setter',
    y: 'difficulty',
    fx: 'year',
    fy: 'route_setter',
    fill: 'type',
    sort: { fy: '-x', reduce: 'count' },
  },
];
for (const grouping of groupings) {
  await display(md`## ${grouping.name}`);
  await display(
    Plot.plot({
      grid: true,
      width: grouping.fx ? baseWidth * 2 : baseWidth,
      color: { legend: true, type: 'categorical', ...ascentTypeColorDomain },
      marks: [Plot.frame(), Plot.barX(detailedAscents.toRecords(), Plot.groupY({ x: 'count' }, grouping))],
      y: { type: 'band', reverse: true },
      document,
    }),
  );
}


## Everything

flash redpoint 7B 7A+ 7A 6C+ 6C 6B+ 6B 6A+ 6A 5+ 5 4+ 4 3+ 3 2 difficulty 0 20 40 60 80 100 120 Frequency →

## By year

flash redpoint 2,022 2,023 2,024 2,025 year 7B 7A+ 7A 6C+ 6C 6B+ 6B 6A+ 6A 5+ 5 4+ 4 3+ 3 2 7B 7A+ 7A 6C+ 6C 6B+ 6B 6A+ 6A 5+ 5 4+ 4 3+ 3 2 7B 7A+ 7A 6C+ 6C 6B+ 6B 6A+ 6A 5+ 5 4+ 4 3+ 3 2 7B 7A+ 7A 6C+ 6C 6B+ 6B 6A+ 6A 5+ 5 4+ 4 3+ 3 2 difficulty 0 10 20 30 40 50 Frequency →

## By route setter

flash redpoint Anton Cuevas Silver Breiktopf Benjamin Schlobohm James Thanawat N.N. Stina Dorwarth Robert Rundin Sebastian Dahlgren Isabelle Fugelstad Sophie Machaczek Scott Mooney Oliver Sjödin Robin Dahlberg Sofia Wideroth Tobias Lindgren Zac Chattaway route_setter 2,022 2,023 2,024 2,025 year 7B 7A+ 7A 6C+ 6C 6B+ 6B 6A+ 6A 5+ 5 4+ 4 3+ 3 2 7B 7A+ 7A 6C+ 6C 6B+ 6B 6A+ 6A 5+ 5 4+ 4 3+ 3 2 7B 7A+ 7A 6C+ 6C 6B+ 6B 6A+ 6A 5+ 5 4+ 4 3+ 3 2 7B 7A+ 7A 6C+ 6C 6B+ 6B 6A+ 6A 5+ 5 4+ 4 3+ 3 2 7B 7A+ 7A 6C+ 6C 6B+ 6B 6A+ 6A 5+ 5 4+ 4 3+ 3 2 7B 7A+ 7A 6C+ 6C 6B+ 6B 6A+ 6A 5+ 5 4+ 4 3+ 3 2 7B 7A+ 7A 6C+ 6C 6B+ 6B 6A+ 6A 5+ 5 4+ 4 3+ 3 2 7B 7A+ 7A 6C+ 6C 6B+ 6B 6A+ 6A 5+ 5 4+ 4 3+ 3 2 7B 7A+ 7A 6C+ 6C 6B+ 6B 6A+ 6A 5+ 5 4+ 4 3+ 3 2 7B 7A+ 7A 6C+ 6C 6B+ 6B 6A+ 6A 5+ 5 4+ 4 3+ 3 2 7B 7A+ 7A 6C+ 6C 6B+ 6B 6A+ 6A 5+ 5 4+ 4 3+ 3 2 7B 7A+ 7A 6C+ 6C 6B+ 6B 6A+ 6A 5+ 5 4+ 4 3+ 3 2 7B 7A+ 7A 6C+ 6C 6B+ 6B 6A+ 6A 5+ 5 4+ 4 3+ 3 2 7B 7A+ 7A 6C+ 6C 6B+ 6B 6A+ 6A 5+ 5 4+ 4 3+ 3 2 7B 7A+ 7A 6C+ 6C 6B+ 6B 6A+ 6A 5+ 5 4+ 4 3+ 3 2 7B 7A+ 7A 6C+ 6C 6B+ 6B 6A+ 6A 5+ 5 4+ 4 3+ 3 2 difficulty 0 5 10 15 0 5 10 15 0 5 10 15 0 5 10 15 0 5 10 15 0 5 10 15 0 5 10 15 0 5 10 15 0 5 10 15 0 5 10 15 0 5 10 15 Frequency →

In [ ]:
await display(
  md`
## Poor man's CPR
  `,
);

Plot.plot(
  (() => {
    const data = detailedAscents
      .groupBy(pl.col('created_at').date.strftime('%Y-%U').alias('week'), 'type')
      .agg(
        pl.col('created_at').first(),
        pl.col('difficulty').min().alias('difficultyMin'),
        pl.col('difficulty').max().alias('difficultyMax'),
        pl.col('numberDifficulty').min().sub(3).alias('numberDifficultyMin'),
        pl.col('numberDifficulty').max().alias('numberDifficultyMax'),
        pl.col('created_at').count().alias('count'),
      );

    return {
      figure: true,
      grid: true,
      color: { legend: true, type: 'categorical', ...ascentTypeColorDomain },
      width: 1500,
      y: {
        ticks: boulderScores.filter((grade) => grade.font <= '7B').map((grade) => grade.score),
        tickFormat: (v) => boulderScores.find(({ score }) => v === score)?.font,
      },
      marks: [
        Plot.frame(),
        ...['redpoint', 'flash'].map((type) =>
          Plot.rectY(
            data.filter(pl.col('type').eq(pl.lit(type))).toRecords(),

            {
              interval: 'week',
              x: 'created_at',

              y1: 'numberDifficultyMin',
              y2: 'numberDifficultyMax',
              fill: 'type',
              title: (d) => {
                if (d.difficultyMin === d.difficultyMax) return d.difficultyMin;
                return `${d.difficultyMin} - ${d.difficultyMax}`;
              },
              inset: 1,
              mixBlendMode: 'color-dodge',
            },
          ),
        ),
        /*
        Plot.rectY(data.toRecords(), { interval: 'week', x: 'created_at', y: 'count', fill: 'gray' }),
        Plot.text(data.toRecords(), {
          x: 'created_at',
          y: 'count',
          text: 'count',
          lineAnchor: 'bottom',
        }),
        */
      ],
      document,
    };
  })(),
);



## Poor man's CPR
  

flash redpoint 2 2+ 3 3+ 4 4+ 5 5+ 6A 6A+ 6B 6B+ 6C 6C+ 7A 7A+ 7B Apr 2022 Jul Oct Jan 2023 Apr Jul Oct Jan 2024 Apr Jul Oct Jan 2025 created_at → 6A 6A+ - 7A+ 6B+ - 7A 6B+ - 7A+ 6C 6B 6B+ 5 - 6C 6C+ - 7A 6A+ - 7A 6B 6A+ - 6C 7A+ - 7B 6B - 6C 6B+ - 7B 6B+ 6B+ - 6C+ 6B - 6C 6B+ - 6C 6A+ - 6B 6B+ - 7A+ 6A+ - 6B+ 5 - 6A 5+ - 7A+ 6C 6B+ - 6C 6B+ - 7A+ 6A+ - 6B 6C+ - 7B 6A+ 6B+ 7A - 7A+ 5+ - 7A 7A+ 4+ 6B+ 5+ - 6B 6C 6C 6B 7A - 7A+ 6A - 6C 6B - 6C+ 6B+ - 7A 6B+ - 6C+ 6B+ - 6C+ 6C 6A+ - 7A 6C 6B+ 6B+ - 7A 6B+ - 6C+ 5+ - 6A 6C - 7A 6C 6B - 7A+ 6C 6B+ 6B+ - 6C 6C 6A - 6C 6A+ - 7A+ 6B - 6C+ 6A+ - 7A 6B - 6C 6A+ - 6B+ 6C - 7A 6C - 7A 6C+ 6C 6B+ - 6C 6C 6A+ - 6B+ 6A - 7A 6B+ - 6C 6C 6A+ - 7A+ 6B - 6C+ 6C 6B 6A+ - 7A 6C+ 6A+ - 7A 6B - 6C 6C - 7A+ 6C 7A 6B+ - 6C 6B 6C - 7B 3 - 5+ 3 - 6A+ 4+ - 6B 3+ - 6A+ 4 - 6A+ 3 - 6C 3 - 6A+ 3+ - 6A+ 3 - 6A 3 - 6C 3+ - 7A 5 - 6A 3 - 6B+ 4+ - 6B 3 - 6B+ 3+ - 6B 4 - 6B 4 - 6A+ 3+ - 6B+ 4 - 6C 3+ - 6B 3+ - 6B+ 5+ - 6C 3+ - 6B+ 3 - 6C+ 3 - 5+ 3 - 6B 3 - 6B 3 - 6C 3 - 6A+ 3 - 6B+ 4+ - 6A+ 3 - 6B+ 3 - 6C 5 - 6B 3 - 6B+ 3 - 5 4+ - 6C+ 6A+ - 6B+ 3+ - 6B 3+ - 6A+ 3 - 6B+ 3 - 6A 3 - 6B+ 3 - 6C 3 - 6B+ 6A - 6A+ 3+ - 6B+ 3 - 6C 6B - 6C+ 3 - 6B+ 2 - 6B+ 4 - 6A+ 6A - 6B 4+ - 6A+ 3 - 6B+ 3+ - 6B 3+ - 6A+ 3+ - 6B+ 4+ 4+ - 6B+ 3 - 6A+ 4 - 6C+ 3+ - 6B+ 3 - 6B+ 5 - 6C 3+ - 6C 3 - 6B+ 3 - 6B 3 - 6A+ 4+ - 6B+ 5+ - 6A+ 6A - 6B+ 4 - 6A+ 4 - 6A+ 5 - 6B+ 3+ - 6B 3+ - 6B+ 6A+ 5 4 - 6C 3+ - 6A+ 6A+ - 6B 3+ - 6C 4+ 3+ - 6B+ 5+ - 6C+ 3 - 6C+ 3+ - 6B+ 3+ - 6A+ 3 - 6B+ 4+ - 6A+ 3 - 6C 3 - 6A+ 3+ - 6B 3+ - 6B 4+ - 6B

In [ ]:
Plot.plot(
  (() => {
    const data = detailedAscents
      .groupBy(pl.col('created_at').date.strftime('%Y-%U').alias('grouped'))
      .agg(
        pl.col('created_at').first().cast(pl.Date).alias('date'),
        pl.col('numberDifficulty').sum().alias('totalNumberDifficulty'),
        pl.col('created_at').count().alias('count'),
        pl.col('difficulty'),
      )
      .toRecords();
    return {
      figure: true,
      color: { scheme: 'YlGn' },
      marks: [
        Plot.barX(data, {
          interval: 'week',
          x: 'date',
          fill: 'totalNumberDifficulty',
          title: (d) =>
            Object.entries(
              d.difficulty.reduce((acc, grade) => {
                if (!acc[grade]) {
                  acc[grade] = 0;
                }
                acc[grade] += 1;
                return acc;
              }, {}),
            )
              .toSorted((a, b) => a[0].localeCompare(b[0]))
              .reduce((acc, ascents) => [...acc, `${ascents[0]} x${ascents[1]}`], [] as string[])
              .join('; '),
        }),
      ],
      document,
    };
  })(),
);


Apr 2022 Jul Oct Jan 2023 Apr Jul Oct Jan 2024 Apr Jul Oct Jan 2025 3+ x1; 4+ x1; 5+ x1; 6A x1; 6A+ x2; 6B+ x1 4 x2; 4+ x3; 5 x1; 5+ x4; 6A x3; 6A+ x2 3 x2; 4+ x1; 5 x1; 5+ x2; 6A x1; 6A+ x3; 6B x3 3+ x1; 4 x1; 4+ x1; 6A x1; 6A+ x2; 6B x2; 6B+ x1; 6C+ x1 3 x1; 3+ x1; 4 x2; 4+ x1; 5 x1; 5+ x2; 6A x2 4 x1; 5+ x1; 6B x1; 6B+ x1; 6C+ x1; 7A x1 6A+ x1; 6C x1; 7A+ x1 3 x1; 3+ x1; 4+ x1; 5 x1; 5+ x1; 6A+ x1 3+ x1; 4+ x3; 6A x8; 6A+ x4 4+ x1; 7A+ x1 3 x1; 3+ x3; 4 x3; 5 x1; 6A x1; 6B x1; 6B+ x2 5+ x1; 6A+ x1; 7A x1; 7A+ x1 3 x1; 3+ x1; 4 x3; 4+ x1; 5 x3; 6A x2; 6A+ x2; 6B x1; 6B+ x2; 6C x2; 6C+ x1; 7A x1 3 x2; 3+ x1; 4 x1; 5 x1; 6A x1; 6A+ x1; 6B x1; 6B+ x2; 6C+ x2; 7A x2 3 x1; 4+ x2; 5+ x1; 6A x1; 6A+ x2; 6B x1; 6C x1 3 x2; 3+ x2; 4 x1; 4+ x2; 5 x2; 5+ x5; 6A x2; 6A+ x1; 6B x2; 6B+ x1; 6C x1 3 x1; 3+ x1; 4 x1; 4+ x2; 5 x2; 5+ x1; 6A x2; 6A+ x2; 6B x1 3 x2; 3+ x2; 4 x2; 5+ x2; 6A x4; 6A+ x2; 6B x1; 6B+ x1 5 x1; 5+ x1; 6A x1; 6A+ x2; 6B x1; 6C x2 4+ x1; 5+ x2; 6A+ x2; 6B x1; 6B+ x1 3+ x1; 4 x1; 4+ x1; 5 x2; 5+ x2; 6A x1; 6A+ x1; 6B+ x1; 6C+ x1 5 x2; 5+ x2; 6A x3; 6A+ x3; 6B x3; 6B+ x2; 6C x1 5+ x1; 6A x1; 6A+ x2; 6B x2; 6B+ x2; 6C x2; 6C+ x1; 7A x1 3+ x2; 4 x1; 5 x1; 6B x2; 6B+ x1; 6C+ x1; 7A x1 4+ x2; 5+ x1; 6A x1; 6A+ x1; 6B+ x1; 6C x1; 7A x1 6A x1; 6A+ x1 3+ x1; 4 x1; 5 x1; 5+ x1; 6A x2; 6A+ x2; 6B x2; 6B+ x1 3+ x1; 6B x1; 6B+ x1 3+ x1; 4 x1; 4+ x1; 5 x1; 5+ x2; 6A x3; 6A+ x3; 6B+ x3; 7A+ x1; 7B x1 3 x1; 5 x1; 6B x2; 6C x1 6A x1; 6A+ x1; 6C x1 3+ x1; 4 x1; 5+ x2; 6A+ x1; 6B x2 3+ x1; 4 x3; 4+ x3; 5 x2; 5+ x2; 6A x3; 6A+ x1; 6B x1; 6B+ x1; 6C x2; 7A+ x1 3+ x1; 4+ x1; 5 x1; 5+ x3; 6A x2; 6A+ x2; 6B x1; 6B+ x3; 6C x3 6A+ x1; 6B x1 4 x1; 4+ x1; 6A x1; 6B x1; 6B+ x3; 6C x2; 6C+ x1; 7B x1 3 x1; 4 x1; 4+ x1; 5 x4; 5+ x2; 6A x3; 6A+ x2; 6B x2; 6B+ x1; 6C x2; 6C+ x3; 7A+ x1 3 x2; 3+ x1; 4 x2; 4+ x2; 5 x2; 6A x3; 6A+ x2; 6B x1; 6B+ x2; 6C+ x1 4 x1; 5 x2; 6A x1; 6B x1; 6B+ x1; 6C x2; 6C+ x1 4+ x1; 6A+ x1; 6B+ x2; 6C x1; 6C+ x1 3 x1; 4 x1; 4+ x1; 5+ x1; 6A x1; 6B x1 6A+ x1; 6B x1; 6B+ x1 6A+ x1; 6B x1 3 x1; 4 x1; 5 x6; 6A x1; 6A+ x2; 6B x3; 6B+ x1; 6C x2; 6C+ x1 3+ x2; 4 x1; 4+ x1; 5+ x1; 6A x1; 6B x1; 6B+ x3; 6C+ x1 5 x1 4+ x3; 5+ x3; 6A x1; 6A+ x1; 6B x2; 6B+ x2; 6C x1 5 x1; 5+ x1; 6B+ x3 6B+ x1 4 x1; 5 x1; 6A x1; 6B+ x1; 6C x1 3+ x1; 4+ x2; 6A+ x1; 6B x1; 7A x1 3 x2; 3+ x1; 4 x2; 4+ x4; 5+ x4; 6A x4; 6A+ x3; 6C x1; 7A x2; 7A+ x1 3 x1; 4 x1; 4+ x1; 5 x1; 6A x1; 6A+ x1; 6B+ x1; 7A x1 4+ x1; 5 x5; 5+ x1; 6A x2; 6A+ x5 4 x4; 4+ x6; 5+ x2; 6A x5; 6A+ x3 3 x2; 3+ x2; 4 x3; 4+ x1; 5 x3; 5+ x2; 6A x1; 6A+ x1; 6B x2; 6B+ x1; 7A+ x1 3+ x1; 4 x1; 6A+ x2; 6B+ x1 3 x1; 4 x1; 4+ x2; 5+ x2; 6A x1; 6A+ x2; 6B x2; 6B+ x1; 6C x2 3+ x1; 4 x1; 4+ x1; 5+ x2; 6A+ x1; 6B x2; 6B+ x1; 7A x1 4+ x1; 5 x2; 5+ x5; 6A x2; 6A+ x3 3 x1; 3+ x2; 4+ x3; 5 x1; 5+ x2; 6A x1; 6A+ x1; 6B x1; 6B+ x1; 6C+ x2; 7A x1; 7B x1 3 x2; 4 x1; 4+ x2; 5+ x1; 6A x1; 6A+ x1; 6C x1 6A x1; 6A+ x1; 6B x1; 6B+ x1; 6C x1 3 x1; 4 x1; 4+ x2; 5+ x1; 6A x2; 6A+ x1; 6C x1 3+ x1; 4 x2; 4+ x1; 5+ x1; 6A x1; 6A+ x1; 6C x2 3 x1; 3+ x2; 4 x2; 4+ x1; 5 x2; 6A x1; 6A+ x1; 6B x1; 6B+ x2; 6C x2 6C x2 3+ x1; 5+ x1; 6B x1; 6B+ x1; 7A+ x1 3+ x1; 4 x1; 5+ x1; 6A+ x1; 6B+ x1; 7A+ x1 3 x2; 4 x1; 5 x4; 5+ x1; 6A x3; 6A+ x3; 6B x2; 6B+ x1; 6C+ x3; 7A x1 3 x1; 4+ x1; 5+ x1; 6A x2; 6A+ x2; 6B x1; 6B+ x2; 6C x3; 6C+ x1 3 x1; 3+ x2; 4 x2; 4+ x3; 5 x1; 5+ x1; 6A x2; 6B x2; 6B+ x3; 6C x4 3 x1; 4 x3; 4+ x4; 5 x1; 5+ x4 3 x1; 4 x2; 4+ x1; 5 x3; 5+ x1; 6A x1 5+ x1; 6A x2; 6B+ x1; 6C x1; 6C+ x1 3+ x1; 4 x2; 4+ x1; 5+ x1; 6A x1; 6A+ x1; 6B+ x2; 6C x1 5 x1; 5+ x2; 6A x2; 6C x1 3+ x2; 4 x2; 5 x1; 5+ x2; 6A x1; 6A+ x5; 6B x6; 6B+ x1; 6C x3; 6C+ x1; 7A x1; 7A+ x1 3 x1; 3+ x1; 4+ x1; 5 x2; 5+ x1; 6A x4; 6A+ x3; 6B x4; 6B+ x1; 6C x2 4+ x2; 6A+ x2 4 x1; 6A x1; 6A+ x2; 6B+ x2; 6C x2; 6C+ x1; 7A x1 6C x1 3 x1; 3+ x2; 4 x1; 4+ x2; 5 x1; 5+ x4; 6A x2; 6A+ x4; 6B x4; 6B+ x4; 6C x1 4 x1; 6A+ x1; 6B+ x2; 6C x1 6A x1; 6B x1; 6B+ x1 3+ x1; 4+ x1; 5 x1; 5+ x1; 6A x3; 6A+ x1; 6B+ x2; 6C x1; 6C+ x1; 7A x3 6B x1; 6B+ x1; 6C x1; 6C+ x1; 7A x1 4+ x2; 6A

In [ ]:
import * as d3 from 'npm:d3';

Plot.plot(
  (() => {
    const data = detailedAscents
      .groupBy(pl.col('created_at').date.strftime('%Y-%U').alias('grouped'))
      .agg(
        pl.col('created_at').first().date.strftime('%Y-%m-%d').alias('date'),
        pl.col('numberDifficulty').sum().alias('totalNumberDifficulty'),
        pl.col('created_at').count().alias('count'),
        pl.col('difficulty'),
      )
      .toRecords();
    return {
      figure: true,
      x: { tickFormat: (d) => `Week ${d + 1}`, tickRotate: 90 },
      y: { tickFormat: '', tickSize: 0 },
      color: { scheme: 'YlGn' },
      marginBottom: 47,
      marks: [
        Plot.cell(data, {
          x: (d) => d3.utcWeek.count(d3.utcYear(new Date(d.date)), new Date(d.date)),
          y: (d) => new Date(d.date).getUTCFullYear(),
          fill: 'totalNumberDifficulty',
          title: (d) =>
            Object.entries(
              d.difficulty.reduce((acc, grade) => {
                if (!acc[grade]) {
                  acc[grade] = 0;
                }
                acc[grade] += 1;
                return acc;
              }, {}),
            )
              .toSorted((a, b) => a[0].localeCompare(b[0]))
              .reduce((acc, ascents) => [...acc, `${ascents[0]} x${ascents[1]}`], [] as string[])
              .join('; '),
          inset: 1,
        }),
      ],
      document,
    };
  })(),
);


2022 2023 2024 2025 Week 1 Week 2 Week 3 Week 4 Week 5 Week 6 Week 7 Week 8 Week 9 Week 10 Week 11 Week 12 Week 13 Week 14 Week 15 Week 16 Week 17 Week 18 Week 19 Week 20 Week 21 Week 22 Week 23 Week 24 Week 25 Week 26 Week 27 Week 28 Week 29 Week 30 Week 31 Week 32 Week 33 Week 34 Week 35 Week 36 Week 37 Week 38 Week 39 Week 40 Week 41 Week 42 Week 43 Week 44 Week 45 Week 46 Week 47 Week 48 Week 49 Week 50 Week 51 Week 52 4+ x1; 5 x2; 5+ x5; 6A x2; 6A+ x3 3+ x1; 4+ x1; 5 x1; 5+ x3; 6A x2; 6A+ x2; 6B x1; 6B+ x3; 6C x3 3 x1; 3+ x1; 4 x2; 4+ x1; 5 x1; 5+ x2; 6A x2 6A x1; 6A+ x1; 6C x1 3+ x1; 4 x2; 4+ x1; 5+ x1; 6A x1; 6A+ x1; 6C x2 3 x2; 3+ x1; 4 x2; 4+ x2; 5 x2; 6A x3; 6A+ x2; 6B x1; 6B+ x2; 6C+ x1 5+ x1; 6A x2; 6B+ x1; 6C x1; 6C+ x1 3 x1; 4 x3; 4+ x4; 5 x1; 5+ x4 3 x2; 4+ x1; 5 x1; 5+ x2; 6A x1; 6A+ x3; 6B x3 4 x1; 6A x1; 6A+ x2; 6B+ x2; 6C x2; 6C+ x1; 7A x1 6A+ x1; 6B x1 4+ x1; 5 x5; 5+ x1; 6A x2; 6A+ x5 4 x1; 5 x1; 6A x1; 6B+ x1; 6C x1 3+ x2; 4 x1; 4+ x1; 5+ x1; 6A x1; 6B x1; 6B+ x3; 6C+ x1 3 x1; 3+ x2; 4+ x3; 5 x1; 5+ x2; 6A x1; 6A+ x1; 6B x1; 6B+ x1; 6C+ x2; 7A x1; 7B x1 6A+ x1; 6C x1; 7A+ x1 3+ x1; 5+ x1; 6B x1; 6B+ x1; 7A+ x1 3+ x1; 4+ x1; 5 x1; 5+ x1; 6A x3; 6A+ x1; 6B+ x2; 6C x1; 6C+ x1; 7A x3 6B x1; 6B+ x1; 6C x1; 6C+ x1; 7A x1 5+ x1; 6A x1; 6A+ x2; 6B x2; 6B+ x2; 6C x2; 6C+ x1; 7A x1 3+ x1; 4 x1; 4+ x1; 5+ x1; 6A+ x2; 6B+ x1; 6C x1 3 x1; 3+ x2; 4 x2; 4+ x3; 5 x1; 5+ x1; 6A x2; 6B x2; 6B+ x3; 6C x4 5 x1; 5+ x1; 6A x1; 6A+ x2; 6B x1; 6C x2 3 x2; 4 x1; 5 x4; 5+ x1; 6A x3; 6A+ x3; 6B x2; 6B+ x1; 6C+ x3; 7A x1 3+ x1; 4+ x1; 5 x1; 5+ x1; 6A x1; 6B+ x1; 6C+ x1 3 x1; 3+ x3; 4 x3; 5 x1; 6A x1; 6B x1; 6B+ x2 3+ x1; 4 x1; 5 x1; 5+ x1; 6A x2; 6A+ x2; 6B x2; 6B+ x1 3 x1; 3+ x2; 4+ x2; 5 x3; 5+ x1; 6A x3; 6A+ x2; 6B x2; 6B+ x1; 6C x2 3 x1; 3+ x2; 4 x2; 4+ x1; 5 x2; 6A x1; 6A+ x1; 6B x1; 6B+ x2; 6C x2 4+ x3; 5+ x3; 6A x1; 6A+ x1; 6B x2; 6B+ x2; 6C x1 4+ x1; 5 x3; 6A x1; 6A+ x1; 6B x3; 6C x1; 7B x1 3 x1; 3+ x1; 4+ x2; 5 x1; 6B+ x3; 6C x1 3 x1; 3+ x1; 4+ x1; 5 x1; 5+ x1; 6A+ x1 3+ x1; 4 x1; 4+ x1; 6A x1; 6A+ x2; 6B x2; 6B+ x1; 6C+ x1 3+ x1; 6B x1; 6B+ x1 3 x2; 3+ x2; 4 x1; 4+ x2; 5 x2; 5+ x5; 6A x2; 6A+ x1; 6B x2; 6B+ x1; 6C x1 3 x2; 3+ x2; 4 x3; 4+ x1; 5 x3; 5+ x2; 6A x1; 6A+ x1; 6B x2; 6B+ x1; 7A+ x1 3 x1; 4 x2; 5 x2; 6A x1; 6A+ x2; 6B x1; 6B+ x1; 6C x1; 6C+ x1; 7A x1 4 x2; 4+ x3; 5 x1; 5+ x4; 6A x3; 6A+ x2 4+ x1; 5+ x2; 6A+ x2; 6B x1; 6B+ x1 3 x1; 4 x1; 5 x6; 6A x1; 6A+ x2; 6B x3; 6B+ x1; 6C x2; 6C+ x1 3+ x1; 4+ x2; 6A+ x1; 6B x1; 7A x1 4+ x2; 6A+ x2 3 x1; 4 x1; 4+ x2; 5+ x1; 6A x2; 6A+ x1; 6C x1 3 x1; 3+ x1; 4 x1; 4+ x1; 5 x1; 6A x1; 6A+ x3; 6B x1; 6B+ x2; 6C+ x1; 7A x1 6A+ x1; 6B x1 3+ x1; 5 x1; 5+ x1; 6A x1; 6A+ x1; 6B x1; 6C x1; 7A+ x1 6B+ x1 3 x2; 4 x1; 4+ x2; 5+ x1; 6A x1; 6A+ x1; 6C x1 3 x2; 3+ x2; 4 x2; 5+ x2; 6A x4; 6A+ x2; 6B x1; 6B+ x1 3 x1; 4+ x1; 5+ x1; 6A x2; 6A+ x2; 6B x1; 6B+ x2; 6C x3; 6C+ x1 3 x1; 4 x1; 4+ x1; 5 x4; 5+ x2; 6A x3; 6A+ x2; 6B x2; 6B+ x1; 6C x2; 6C+ x3; 7A+ x1 3 x1; 3+ x1; 4+ x1; 5 x2; 5+ x1; 6A x4; 6A+ x3; 6B x4; 6B+ x1; 6C x2 3 x2; 3+ x1; 4 x1; 4+ x2; 5 x1; 6A x2; 6A+ x1; 6B x1; 6B+ x2; 6C+ x1; 7A+ x1 3 x2; 3+ x1; 4 x1; 5 x1; 6A x1; 6A+ x1; 6B x1; 6B+ x2; 6C+ x2; 7A x2 3+ x1; 4 x1; 6A+ x2; 6B+ x1 4 x1; 4+ x1; 6A x1; 6B x1; 6B+ x3; 6C x2; 6C+ x1; 7B x1 3+ x1; 4 x1; 4+ x1; 5 x2; 5+ x2; 6A x1; 6A+ x1; 6B+ x1; 6C+ x1 3+ x2; 4 x1; 5 x1; 6B x2; 6B+ x1; 6C+ x1; 7A x1 6A x1; 6A+ x1 4+ x1; 5+ x1; 6A x1; 6B+ x1; 6C x1 3 x1; 4 x2; 4+ x1; 5 x3; 5+ x1; 6A x1 3+ x1; 4 x1; 5+ x2; 6A+ x1; 6B x2 3+ x1; 4 x2; 4+ x1; 5+ x1; 6A x1; 6A+ x1; 6B+ x2; 6C x1 4+ x2; 6A x3; 6A+ x1; 6B x2; 6C x1; 6C+ x1 3+ x1; 4 x1; 4+ x1; 5+ x2; 6A+ x1; 6B x2; 6B+ x1; 7A x1 6A x1; 6B x1; 6B+ x1 4+ x1; 7A+ x1 6C x2 5+ x1; 6A+ x1; 7A x1; 7A+ x1 3 x1; 5 x1; 6B x2; 6C x1 3 x1; 4 x1; 4+ x1; 5+ x1; 6A x1; 6B x1 3+ x1; 4+ x3; 6A x8; 6A+ x4 6A+ x1; 6B x1; 6B+ x1 3 x1; 4+ x2; 5+ x1; 6A x1; 6A+ x2; 6B x1; 6C x1 3 x1; 3+ x1; 4 x1; 4+ x2; 5 x2; 5+ x1; 6A x2; 6A+ x2; 6B x1 4 x4; 4+ x6; 5+ x2; 6A x5; 6A+ x3 3 x2; 3+ x1; 4 x2; 4+ x4; 5+ x4; 6A x4; 6A+ x3;